In [ ]:
# Install required packages
!pip install xgboost pandas numpy scikit-learn matplotlib seaborn

# Create our training script
import pandas as pd
import numpy as np
import boto3
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

print("Setup complete!")

In [ ]:
# Generate synthetic battery data (paste your BatteryFailurePredictionModel class here)
model = BatteryFailurePredictionModel()
df = model.generate_synthetic_data(10000)

# Display data overview
print("Dataset shape:", df.shape)
print("\nFeature overview:")
print(df.describe())
print("\nFailure rate:", df['will_fail'].mean())

# Visualize data
plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
sns.histplot(df['battery_level'], bins=30)
plt.title('Battery Level Distribution')

plt.subplot(2, 3, 2)
sns.histplot(df['temperature'], bins=30)
plt.title('Temperature Distribution')

plt.subplot(2, 3, 3)
sns.countplot(data=df, x='will_fail')
plt.title('Failure Labels')

plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for training
X = df[model.feature_columns]
y = df['will_fail']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
X_train_scaled = model.scaler.fit_transform(X_train)
X_test_scaled = model.scaler.transform(X_test)

# Train model
model.model = RandomForestClassifier(n_estimators=100, random_state=42)
model.model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = model.model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
import tarfile
import os

# Get SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

print(f"Using bucket: {bucket}")
print(f"Using role: {role}")

In [ ]:
# Save model and scaler
joblib.dump(model.model, 'battery_failure_model.pkl')
joblib.dump(model.scaler, 'battery_scaler.pkl')

# Create model artifact archive
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('battery_failure_model.pkl')
    tar.add('battery_scaler.pkl')

# Upload to S3
model_artifacts = sagemaker_session.upload_data(
    path='model.tar.gz',
    bucket=bucket,
    key_prefix='battery-failure-model'
)

print(f"Model uploaded to: {model_artifacts}")

In [ ]:
%%writefile inference.py
import joblib
import pandas as pd
import numpy as np
import json

def model_fn(model_dir):
    """Load model from S3"""
    model = joblib.load(f"{model_dir}/battery_failure_model.pkl")
    scaler = joblib.load(f"{model_dir}/battery_scaler.pkl")
    return {'model': model, 'scaler': scaler}

def predict_fn(input_data, model_dict):
    """Make predictions"""
    model = model_dict['model']
    scaler = model_dict['scaler']
    
    # Convert to DataFrame
    df = pd.DataFrame([input_data])
    
    # Scale features
    features = ['battery_level', 'temperature', 'humidity', 'robot_age_days',
                'daily_hours', 'charge_cycles', 'robot_type_encoded']
    
    X_scaled = scaler.transform(df[features])
    
    # Predict
    prediction = model.predict(X_scaled)[0]
    probability = model.predict_proba(X_scaled)[0][1]
    
    return {
        'will_fail': int(prediction),
        'failure_probability': float(probability),
        'risk_level': 'HIGH' if probability > 0.7 else 'MEDIUM' if probability > 0.3 else 'LOW'
    }

def input_fn(request_body, request_content_type):
    """Parse input"""
    if request_content_type == 'application/json':
        return json.loads(request_body)
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def output_fn(prediction, accept):
    """Format output"""
    if accept == 'application/json':
        return json.dumps(prediction), accept
    raise ValueError(f"Unsupported accept type: {accept}")

In [ ]:
# Create SageMaker model
sklearn_model = SKLearnModel(
    model_data=model_artifacts,
    role=role,
    entry_point='inference.py',
    framework_version='0.23-1',
    py_version='py3'
)

# Deploy endpoint
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='battery-failure-predictor'
)

print("✅ Model deployed successfully!")
print(f"Endpoint name: battery-failure-predictor")

In [ ]:
# Test prediction
test_data = {
    'battery_level': 25,
    'temperature': 65,
    'humidity': 70,
    'robot_age_days': 500,
    'daily_hours': 18,
    'charge_cycles': 300,
    'robot_type_encoded': 0  # assembly robot
}

# Make prediction
result = predictor.predict(test_data)
print("🔮 Prediction Result:")
print(json.dumps(result, indent=2))